In [1]:
import ee 
from ee import batch

# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()
from make_clim import make_climate

from stack_images_qgis import stacking
from mask_image import define_image
from stratitication import make_strata
from stratitication import prop_allocation
from change_no_change import suit_layer 

from add_neigh import add_neigh
from add_distance import add_distance
#from change_no_change import suit_layer 
import geemap

#lc_all = define_image(0,0,'none',2010,5)

#parameters to change
# tmov threshold - 40, 50, 60 

val_year = 2015
train_year = 2010
i_year = 1983
lc_img = define_image(50,0,'tmov',train_year,5) #temporal mov threshold >= 40, 

lim_temp_f = ee.FeatureCollection('users/mioash/aust_cd66states')
ucl = ee.FeatureCollection('users/mioash/SUA_2016_AUST')
vic = lim_temp_f.filterMetadata('STE', 'equals', 6)
geom = vic

#################CHANGE ACCESSIBILITY

fst_img = stacking(train_year-10,train_year-5,'mean',False,False) # initial year, end year, reducer for climate, get climate, get precipitation
bfst = fst_img.bandNames().getInfo()

clm_mean = stacking(train_year-10,train_year-5,'p50',True,True)
clm_lterm = stacking(i_year,train_year-5,'p50',True,True)
clm_anomaly = clm_mean.subtract(clm_lterm)

bn = clm_anomaly.bandNames().getInfo()
bn = ["{}{}".format(i,str('_anom')) for i in bn]
clm_anomaly = clm_anomaly.rename(bn) 

roads = ee.FeatureCollection('users/verdemuskuna/aus_roads/gis_osm_roads_free_1')
roads = roads.filter(ee.Filter.inList('fclass', ['motorway','trunk','primary','secondary','unclassified','tertiary','residential','living_street']))
rd = ee.Image(0).paint(roads,'code').updateMask(fst_img.select('accessibility'))
rd = rd.updateMask(rd.neq(0))
dist_roads = rd.select('constant').fastDistanceTransform(1000).sqrt().multiply(ee.Image.pixelArea().sqrt())#//.divide(1000000)//.clip(to_clip)
dist_roads = dist_roads.rename('dist_roads').updateMask(fst_img.select('accessibility').add(100))
#dist_roads = dist_roads.expression(
#  '(1 + (1/dd))',{
#  'dd': dist_roads.select(['dist_roads'])
#}).rename('dist_roads')

wways = ee.FeatureCollection('users/verdemuskuna/aus_roads/gis_osm_waterways_free_1')
wways = wways.filter(ee.Filter.inList('fclass', ['river']))
ww = ee.Image(0).paint(wways,'code').updateMask(fst_img.select('accessibility'))
ww = ww.updateMask(ww.neq(0))
dist_wways = ww.select('constant').fastDistanceTransform(1000).sqrt().multiply(ee.Image.pixelArea().sqrt())#//.divide(1000000)//.clip(to_clip)
dist_wways = dist_wways.rename('dist_wways').updateMask(fst_img.select('accessibility').add(100))
#dist_wways = dist_wways.expression(
#  '(1 + (1/dd))',{
#  'dd': dist_wways.select(['dist_wways'])
#}).rename('dist_wways')


In [2]:
img_for_train = lc_img.addBands(fst_img).addBands(clm_mean).addBands(clm_anomaly).addBands(dist_roads).addBands(dist_wways)

idist = img_for_train.select('lc_t1').rename('lc_temp')
lcclasses = [0,1,2,3,4,5]
cpixels = [10,10,10,10,10]

#imgn = img_for_train

#idist = img_for_train.select('dist_roads')
for i in range(0,5):
    idist = add_distance(idist,lcclasses[i],'lc_temp',geom,cpixels[i])
    

nat_npa = ee.Image("users/mioash/drivers_lcaus/natural_areas/nat_npa_ready")
nat_npa = nat_npa.select('npa')
to_test = ee.Image(0).clip(geom).where(nat_npa,1)
to_test = to_test.rename('npa')

distto_npa = add_distance(to_test,1,'npa',geom,1)
distto_npa = distto_npa.rename(['npa','distto_npa'])

idist = idist.addBands(distto_npa).addBands(dist_roads).addBands(dist_wways)

ineigh = img_for_train.select('dist_wways')

for i in range(3,213,6):
    ineigh = ineigh.addBands(add_neigh(lc_img.select('lc_t1'),i,1,True))

#img_for_train = img_for_train.addBands(distto_npa.select('distto_npa'))
#print(idist.bandNames().getInfo())          

In [3]:
lc_cnc_c = suit_layer(lc_img,'binary',True,0)#c
lc_cnc_f = suit_layer(lc_img,'binary',True,1)#f
lc_cnc_g = suit_layer(lc_img,'binary',True,2)#g
lc_cnc_u = suit_layer(lc_img,'binary',True,3)#u

bfst = fst_img.bandNames().getInfo() #first image
blci = lc_img.bandNames().getInfo() #land cover
bclm = clm_mean.bandNames().getInfo() #climate
bcla = clm_anomaly.bandNames().getInfo() #anomaly
broads = dist_roads.bandNames().getInfo() #distance to roads
bwways = dist_wways.bandNames().getInfo() #distance to water ways
bdist = idist.bandNames().getInfo() #
bneigh = ineigh.bandNames().getInfo()

In [4]:
lc_imgb = lc_cnc_c
lcclass = 'crop'

strat = make_strata(lc_imgb,2,True) # binary
ssamples = prop_allocation(ee.Image(strat).toInt(),geom,5000)

one_band = lc_imgb.add(1)
lc1 = one_band.select('lc_t1').multiply(10000)#.add()
lc2 = one_band.select('lc_t2').multiply(1000)
lc3 = one_band.select('lc_t3').multiply(100)
lc4 = one_band.select('lc_t4').multiply(10)
lc5 = one_band.select('lc_t5').multiply(1)
lc_past = lc1.add(lc2).add(lc3).add(lc4).add(lc5).rename('lc_past')

lc_imgb = lc_imgb.select('lc_dep').rename('lc_dep_bin')
lc_imgb = lc_imgb.addBands(lc_past)

proj = lc_img.projection()

pop = ee.ImageCollection("projects/sat-io/open-datasets/hrslpop").median().clip(geom)
pop = pop.unmask(0).reproject(proj.atScale(30)).clip(geom)
pop = pop.select('b1').rename('pop')
#Map.addLayer(lc_imgb.clip(geom),'', 'cnc1')
lc_all_a= ee.Image("users/mioash/Calderon_etal_Australian_land-cover/Lc_Aus_1985_2015_v1")
img_for_train = img_for_train.addBands(idist).addBands(pop).addBands(lc_all_a.select('b2015')).addBands(ineigh).addBands(distto_npa.select('distto_npa'))

#print(lc_change.bandNames().getInfo())          

In [10]:
#print(lc_imgb.bandNames().getInfo()+bfst+blci+bclm+bcla+bdist+
#          distto_npa.select('distto_npa').bandNames().getInfo()+
#          lc_all_a.select('b2015').bandNames().getInfo()+pop.bandNames().getInfo())

roi_australia = ee.Geometry.Rectangle(112, -43.95, 155, -9)

ee.batch.Export.image.toCloudStorage(image=idist.toFloat(),description='distances_2010',bucket='test_lc',scale=30,
                                    maxPixels=1e13,region=roi_australia ).start()


In [11]:
lc_change = lc_imgb.addBands(img_for_train)
bbands = [lc_imgb.bandNames().getInfo()+bfst+lc_all_a.select('b2015').bandNames().getInfo(), #0
          pop.bandNames().getInfo(),#1
          bdist+distto_npa.select('distto_npa').bandNames().getInfo(),#2
          broads+bwways,#3
          blci+bclm+bcla,#4
          bneigh]#5
def myproperties(feature):
  feature=ee.Feature(feature).setGeometry(None)
  return feature

for i in range(0,len(bbands)):
    tvals = lc_change.select(bbands[i]).addBands(ee.Image.pixelLonLat().float()).sampleRegions(
                            collection= ssamples, 
                            scale= 30,
                            tileScale=16)
    tvals = tvals.map(myproperties)
    ee.batch.Export.table.toDrive(collection=tvals,description='tas_'+lcclass+'_'+str(i)+'_5000p').start()